<a href="https://colab.research.google.com/github/JoelAbadi/DI-Bootcamp/blob/main/W3_D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 1

In [22]:
from google.colab import files

# Upload file manually (upload train.csv)
uploaded = files.upload()



Saving titanic dataset.zip to titanic dataset.zip


In [25]:
import zipfile

with zipfile.ZipFile("titanic dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("titanic_data")  # Extract to a simple folder



In [26]:
import os

# List all files under the extracted folder
for root, dirs, files in os.walk("titanic_data"):
    for file in files:
        print(os.path.join(root, file))


titanic_data/titanic dataset/gender_submission.csv
titanic_data/titanic dataset/test.csv
titanic_data/titanic dataset/train.csv


In [27]:
import pandas as pd

# Update this path based on the output above
file_path = "titanic_data/titanic dataset/train.csv"

df = pd.read_csv(file_path)

# Check for duplicates
print("Number of rows before removing duplicates:", len(df))
print("Number of duplicate rows:", df.duplicated().sum())

# Remove duplicates
df_cleaned = df.drop_duplicates()
print("Number of rows after removing duplicates:", len(df_cleaned))


Number of rows before removing duplicates: 891
Number of duplicate rows: 0
Number of rows after removing duplicates: 891


Exercise 2

In [28]:
# 1. Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# 2. Drop rows with too many missing values (if needed)
# Example: drop rows where all elements are NaN
df_dropped = df.dropna(how='all')

# 3. Fill missing Age values with the median
df_filled = df_dropped.copy()
df_filled['Age'] = df_filled['Age'].fillna(df_filled['Age'].median())

# 4. Fill Embarked with the most common value (mode)
df_filled['Embarked'] = df_filled['Embarked'].fillna(df_filled['Embarked'].mode()[0])

# 5. Drop Cabin column (too many missing values)
df_filled = df_filled.drop(columns=['Cabin'])

# 6. Confirm there are no missing values left
print("\nMissing values after handling:")
print(df_filled.isnull().sum())


Missing values per column:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Missing values after handling:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


Exercise 3


In [29]:
df_filled['FamilySize'] = df_filled['SibSp'] + df_filled['Parch'] + 1


In [30]:
df_filled['Title'] = df_filled['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [31]:
# Map common titles and group rare ones under 'Rare'
title_map = {
    'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
    'Dr': 'Rare', 'Rev': 'Rare', 'Col': 'Rare', 'Major': 'Rare', 'Mlle': 'Miss',
    'Countess': 'Rare', 'Ms': 'Miss', 'Lady': 'Rare', 'Jonkheer': 'Rare',
    'Don': 'Rare', 'Dona': 'Rare', 'Mme': 'Mrs', 'Capt': 'Rare', 'Sir': 'Rare'
}
df_filled['Title'] = df_filled['Title'].map(title_map)


In [32]:
# One-hot encode selected categorical features
df_encoded = pd.get_dummies(df_filled, columns=['Sex', 'Embarked', 'Title'], drop_first=True)


In [33]:
from sklearn.preprocessing import StandardScaler

# Create a scaler
scaler = StandardScaler()

# List of numerical features to scale
num_features = ['Age', 'Fare', 'FamilySize']

# Apply scaler
df_encoded[num_features] = scaler.fit_transform(df_encoded[num_features])


In [34]:
from google.colab import files

# Upload the kaggle.json file
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [35]:
import os
import zipfile

# Move kaggle.json to correct directory
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/kaggle.json

# Set correct permissions
!chmod 600 /root/.kaggle/kaggle.json

# Download the Titanic dataset
!kaggle competitions download -c titanic


  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 80.6MB/s]


In [36]:
# Unzip the downloaded file
with zipfile.ZipFile("titanic.zip", 'r') as zip_ref:
    zip_ref.extractall("titanic_kaggle")

# Read the CSV
import pandas as pd
df_kaggle = pd.read_csv("titanic_kaggle/train.csv")

# Show the first few rows
df_kaggle.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Exercise 4

In [37]:
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

# Detect outliers in Age and Fare
outliers_age = detect_outliers_iqr(df_kaggle, 'Age')
outliers_fare = detect_outliers_iqr(df_kaggle, 'Fare')

print(f"Outliers in Age: {len(outliers_age)}")
print(f"Outliers in Fare: {len(outliers_fare)}")


Outliers in Age: 11
Outliers in Fare: 116


In [38]:
def cap_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[column] = data[column].clip(lower_bound, upper_bound)
    return data

# Cap outliers
df_kaggle = cap_outliers_iqr(df_kaggle, 'Age')
df_kaggle = cap_outliers_iqr(df_kaggle, 'Fare')


In [39]:
from scipy import stats
import numpy as np

# Calculate Z-scores
z_scores = np.abs(stats.zscore(df_kaggle[['Age', 'Fare']].dropna()))

# Detect Z-score outliers
threshold = 3
outliers_z = (z_scores > threshold).sum(axis=0)
print(f"Z-score Outliers — Age: {outliers_z[0]}, Fare: {outliers_z[1]}")


Z-score Outliers — Age: 0, Fare: 0


Exercise 5


In [40]:
# Check basic stats
df_kaggle[['Age', 'Fare']].describe()


,Age,Fare
count,714.000000,891.000000
mean,29.622700,24.046813
std,14.316665,20.481625
min,0.420000,0.000000
25%,20.125000,7.910400
50%,28.000000,14.454200
75%,38.000000,31.000000
max,64.812500,65.634400


In [41]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_standardized = df_kaggle.copy()

# Columns to standardize
standard_cols = ['Age', 'Fare']

df_standardized[standard_cols] = scaler.fit_transform(df_standardized[standard_cols])


In [42]:
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()
df_normalized = df_kaggle.copy()

# Columns to normalize
normalize_cols = ['Age', 'Fare']

df_normalized[normalize_cols] = normalizer.fit_transform(df_normalized[normalize_cols])


In [43]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Create a copy of the dataset to preserve the original
df_normalized = df_kaggle.copy()

# Select columns to normalize
columns_to_normalize = ['Age', 'Fare']  # Add 'FamilySize' if you created it

# Apply MinMaxScaler
df_normalized[columns_to_normalize] = scaler.fit_transform(df_normalized[columns_to_normalize])


In [44]:
# Confirm values are in [0, 1] range
df_normalized[columns_to_normalize].describe()


,Age,Fare
count,714.000000,891.000000
mean,0.453511,0.366375
std,0.222334,0.312056
min,0.000000,0.000000
25%,0.306014,0.120522
50%,0.428311,0.220223
75%,0.583608,0.472313
max,1.000000,1.000000


Exercise 6


In [45]:
# View object (categorical) columns
df_normalized.select_dtypes(include='object').columns


Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [46]:
# Apply one-hot encoding (drop_first avoids dummy variable trap)
df_encoded = pd.get_dummies(df_normalized, columns=['Sex', 'Embarked'], drop_first=True)


In [48]:
# Make sure 'Title' exists by extracting it from 'Name'
df_encoded = df_kaggle.copy()  # or df_normalized, if you're working from the normalized set

df_encoded['Title'] = df_encoded['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Optional: group rare titles
title_map = {
    'Mr': 'Mr', 'Miss': 'Miss', 'Mrs': 'Mrs', 'Master': 'Master',
    'Dr': 'Rare', 'Rev': 'Rare', 'Col': 'Rare', 'Major': 'Rare',
    'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Don': 'Rare', 'Dona': 'Rare',
    'Lady': 'Rare', 'Countess': 'Rare', 'Capt': 'Rare', 'Sir': 'Rare', 'Jonkheer': 'Rare'
}
df_encoded['Title'] = df_encoded['Title'].map(title_map)



In [49]:
df_encoded = pd.get_dummies(df_encoded, columns=['Title'], drop_first=True)


Exercise 7


In [50]:
import pandas as pd

# Example bins: Child, Teen, Young Adult, Adult, Senior
age_bins = [0, 12, 18, 35, 60, 120]
age_labels = ['Child', 'Teen', 'Young_Adult', 'Adult', 'Senior']

# Create AgeGroup column
df_encoded['AgeGroup'] = pd.cut(df_encoded['Age'], bins=age_bins, labels=age_labels)


In [51]:
# One-hot encode AgeGroup
df_encoded = pd.get_dummies(df_encoded, columns=['AgeGroup'], drop_first=True)


In [52]:
df_encoded[['Age', 'AgeGroup_Young_Adult', 'AgeGroup_Adult']].head()


,Age,AgeGroup_Young_Adult,AgeGroup_Adult
0,22.0,True,False
1,38.0,False,True
2,26.0,True,False
3,35.0,True,False
4,35.0,True,False
